In [110]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
     
     
     

### Community Health Benefit Provision by Hospitals

This is the project that we examine how community benefits provisions by hospitals vary over hospital-insurer market share and market power. There are three projects under the main project.

- First project studies how community benefits provision changes by hospital market share using the data from AHA and IRS 990 form.
- Second project adds insurers market shares and examines whether they affect the ability of provision of hospitals. The new data is Interstudy data.
- Third projects studies same question using MA all payer claims data.

For this project I use only pyhton for data collection, cleaning and organizing. For data analysis, I use Pandas, R and Stata. All codes and results will store in https://github.com/msari6/CommunityHealth repository on a timely base once I finish codes for each project.

##### Notes on Data
- American Health Association Data cover AHA Annual Survey of Hospitals from 2010 to 2017.
- IRS Form 990 data covers from 2010 to 2017. 

In [58]:
import pandas as pd
import numpy as np
import os
from os.path import join
import zipfile 
from urllib.request import urlopen
from io import BytesIO
## Steps to take
## 1. first append aha data from 2012 to 2016
## 2. then, merge aha data with NPI data if necessary
## 3. then merge the merged data with IRS on name and address.

# 1. 
    # Select variables that I needs from data dictionary. But since the dataset is not too big it might be better to append all years first
d = {}
d[2017] = pd.read_csv("/home/msari/Project1/RawData/AHA/2017/FY2017-ASDB/COMMA/ASPUB17.CSV", low_memory= False) 
d[2017] = d[2017].assign(Year = 2017)
d[2016] = pd.read_csv("/home/msari/Project1/RawData/AHA/2016/FY2016-Annual-Survey-Database/COMMA/ASPUB16.CSV", low_memory = False)
d[2016] = d[2016].assign(Year = 2016)
d[2015] = pd.read_csv("/home/msari/Project1/RawData/AHA/2015/FY2015-annual-survey-database/COMMA/ASPUB15.CSV", encoding = "ISO-8859-1", low_memory = False)
d[2015] = d[2015].assign(Year = 2015)
d[2014] = pd.read_csv("/home/msari/Project1/RawData/AHA/2014/FY2014-ASDB/COMMA/ASPUB14.CSV", encoding = "ISO-8859-1", low_memory = False)
d[2014] = d[2014].assign(Year = 2014)
d[2013] = pd.read_csv("/home/msari/Project1/RawData/AHA/2013/FY2013_ASDB/COMMA/ASPUB13.CSV", encoding = "ISO-8859-1", low_memory = False)
d[2013] = d[2013].assign(Year = 2013)
d[2012] = pd.read_csv("/home/msari/Project1/RawData/AHA/2012/FY2012_ASDB/COMMA/ASPUB12.csv", encoding = "ISO-8859-1", low_memory = False)
d[2012] = d[2012].assign(Year = 2012)
d[2011] = pd.read_csv("/home/msari/Project1/RawData/AHA/2011/FY2011_ASDB/COMMA/ASPUB11.csv", encoding = "ISO-8859-1", low_memory = False)
d[2011] = d[2011].assign(Year = 2011)
d[2010] = pd.read_csv("/home/msari/Project1/RawData/AHA/2010/FY2010_ASDB/COMMA/ASPUB10.csv", encoding = "ISO-8859-1", low_memory = False)
d[2010] = d[2010].assign(Year = 2010)

#merge the data
AHA = pd.concat([d[2010], d[2011], d[2012] , d[2013], d[2014], d[2015], d[2016], d[2017]], sort = False)
# set indexes
AHA1 = AHA.set_index(['ID', 'NPINUM', 'MCRNUM', 'Year'])
#save the file as csv to use it later
AHA1.to_csv("/home/msari/Project1/Data/AHA.csv", index = True)
print("The shape of AHA data from 2010 to 2017 is ", AHA.shape)
print("The unique number of hospitals in the dataset is ", len(AHA['ID'].value_counts())) 
print("Note: some of these hospitals were closed or merged to another hospitals.")

The shape of AHA data from 2010 to 2017 is  (50264, 1249)
The unique number of hospitals in the dataset is  6804
Note: some of these hospitals were closed or merged to another hospitals.


In [69]:
AHA = pd.concat([d[2010], d[2011], d[2012] , d[2013], d[2014], d[2015], d[2016], d[2017]], sort = False)
# set indexes
AHA1 = AHA.set_index(['ID', 'NPINUM', 'MCRNUM', 'Year'])
#save the file as csv to use it later
AHA1.to_csv("/home/msari/Project1/Data/AHA.csv", index = True)

#### The selected variables from AHA data

Here is a brief list for variables that I selected from AHA data for descriptive analysis. 

- 'ID': AHA identification number
- 'NPINUM': National Provider Number
- 'MCRNUM': Medicare Provider ID
- 'MNAME': Hospital name
- 'SYSNAME': System name
- 'SYSID': Health care system ID
- 'STCD': AHA State Code
- 'CNTRL': Control Code 
- 'SERV' : Service Code
- 'DTBEG': Beginning of reporting period
- 'DTEND': End of reporting period
- 'FISYR': Fiscal Year
- 'SERV': Service Code
- 'HSACODE': Health Service Area Code - Dartmouth
- 'HSANAME': Health Service Area Name - Dartmouth
- 'HRRNAME': Health Referral Region Name Dartmouth
- 'HRRCODE': Health Referral Region Code - Dartmouth
- 'MLOCADDR': Hospital Street address
- 'MLOCCITY': City
- 'MLOCSTD': State Code
- 'MSTATE' : Hospital State
- 'MLOCZIP' : Zip Code
- 'SYSADDR' : System address
- 'SYSCITY' : System city
- 'SYSST' : System state
- 'SYSZIP' : System ZIP code
- 'SYSAREA' : System area code
- 'SYSTELN' : System telephone number
- 'SYSTEM_PRIMARY_CONTACT': System primary contact
- 'SYSTITLE' : System contact's title
- 'LAT' : Hospital, Latitude
- 'LONG' : Hospital, Longitude
- 'CNTYNAME' : County Name, State Abbreviation
- 'CBSANAME' : Core-Based Statistical Area Name, State Abbreviation
- 'CBSATYPE' : Core-Based Statistical Area Type
- 'CBSACODE' : Core-Based Statistical Area Code
- 'DIVNAME' : Metropolitan Division name
- 'DIVCODE' : Metropolitan Division code
- 'CSANAME' : Combined Statistical Area name
- 'CSACODE' : Combined Statistical Area code
- 'MCNTYCD' : Modified FIPS County Code
- 'FCOUNTY' : FIPS State and County Code
- 'FSTCD' : FIPS State code
- 'FCNTYCD': FIPS County code
- 'INSPRD' : Hospital/system offers insurance products (via ownership or joint venture)
- 'INSPT' : Hospital/system partners with insurer to offer insurance products
- 'LBEDSA' : Licensed beds total facility
- 'BDTOT' : Total facility beds set up and staffed at the end of reporting period
- 'IPDTOT' : Total facility inpatient days
- 'MAPP8' : Member of Council of Teaching Hospital of the Association of American Medical Colleges (COTH)
- 'CHC' :  Community Hospital code
- 'MHSMEMB': System Member
- 'PHYGP': Is hospital owned in whole or in part by physicians or a physician group?
- 'CBSATYPE' : Core-Based Statistical Area Type 

***Note for MAPP8***: AHA documentation considers hospital a major teaching hospital if it has Council of Teaching Hospital designation.

Now I will create a subset of AHA using these variables. 

In [70]:
import warnings
warnings.filterwarnings('ignore')
## now let's create a subset of AHA consisting of selected variables listed above. 
id_variables = ['ID', 'NPINUM', 'MCRNUM', 'MNAME', 'Year', 'SYSNAME', 'SYSID', 'STCD', 'CNTRL', 'SERV', 'DTBEG', 'DTEND', 'FISYR', 'SERV', 'HSACODE', 'HSANAME', 'HRRNAME', 'HRRCODE', 'MLOCADDR', 'MLOCCITY', 'MLOCSTD', 'MSTATE', 'MLOCZIP', 'SYSADDR', 'SYSCITY', 'SYSST', 'SYSZIP', 'SYSAREA', 'SYSTELN', 'SYSTEM_PRIMARY_CONTACT', 'SYSTITLE', 'LAT', 'LONG', 'CNTYNAME', 'CBSANAME', 'CBSATYPE', 'CBSACODE', 'DIVNAME', 'DIVCODE', 'CSANAME', 'CSACODE', 'MCNTYCD', 'FCOUNTY', 'FSTCD', 'FCNTYCD', 'INSPRD', 'INSPT', 'LBEDSA', 'BDTOT', 'IPDTOT', 'MAPP8', 'CHC', 'MHSMEMB', 'PHYGP', 'CBSATYPE']
aha_subset = AHA.loc[:, id_variables]
print("The data with selected variable structure is ", aha_subset.shape)
print("The unique number of hospitals in subset dataset is ", len(aha_subset['ID'].value_counts())) 

The data with selected variable structure is  (50264, 55)
The unique number of hospitals in subset dataset is  6804


#### Market Share of Hospitals
The unit level of AHA dataset is hospital. To note that some hospitals are part of a system while some are not. For market share calculation, I will use herfindahl-hirschman index:
$$HHI_{it}=\sum_{i=1}^N s^2_{it}$$

- s: market share of hospital in market
- N: number of hospitals in the market
- i: one unit of hospital
- t: time

for market share of each hospital is calculated as follows:

$$s_{it}=q_{it} / \sum_{j=1}^N q_{jt}$$

higher HHI for a market means more market concentration (HHI=10000 means monopoly). 
HSA and counties will be market boundaries for preliminary purposes. 
The product of market share, $q$, will be staffed beds and inpatient days.

In [71]:
#Market Calculation for Staffed Beds at HSA level
#aha_subset.groupby(['HSACODE','Year']).BDTOT.apply(lambda x: (x.iloc[0]/x.sum())**2 + (x.iloc[1]/x.sum())**2)
Formul = lambda x: sum([((x.iloc[i]/x.sum())**2)*10000 for i in range(len(x))])
s = aha_subset.groupby(['HSACODE','Year']).BDTOT.apply(Formul)
# now I need to merge the new column "HHI" to main dataframe. 
df = s.reset_index()
df = df.rename(columns = {'BDTOT' : 'HHI_HSA_BED'})
df

,HSACODE,Year,HHI_HSA_BED
0,1001.0,2010,7838.243573
1,1001.0,2011,8699.202895
2,1001.0,2012,6781.443423
3,1001.0,2013,7126.654064
4,1001.0,2014,6827.480519
5,1001.0,2015,8854.848305
6,1001.0,2016,7000.337125
7,1001.0,2017,5636.835330
8,1002.0,2010,10000.000000
9,1002.0,2011,10000.000000


In [72]:
#Market Share Calculation for Patient Days at HSA level
Formul = lambda x: sum([((x.iloc[i]/x.sum())**2)*10000 for i in range(len(x))])
a = aha_subset.groupby(['HSACODE','Year']).IPDTOT.apply(Formul)
# now I need to merge the new column "HHI" to main dataframe. 
a = a.reset_index()
a = a.rename(columns = {'IPDTOT' : 'HHI_HSA_PDAYS'})
a

,HSACODE,Year,HHI_HSA_PDAYS
0,1001.0,2010,9078.699404
1,1001.0,2011,9336.965221
2,1001.0,2012,8873.871035
3,1001.0,2013,8701.725407
4,1001.0,2014,8941.779014
5,1001.0,2015,8956.716728
6,1001.0,2016,8935.451836
7,1001.0,2017,7823.233266
8,1002.0,2010,10000.000000
9,1002.0,2011,10000.000000


In [73]:
print(" the number of HSA is", len(aha_subset['HSACODE'].value_counts()))

 the number of HSA is 3294


The number of HSA didn't look correct before. According to the [National Cancer Institute website](https://seer.cancer.gov/seerstat/variables/countyattribs/Health.Service.Areas.xls) There should be around 3200 Health Service Areas. Eyeballing of data shows that AHA 2011 counts double in the data. I fixed it by changing data type of HSACODE in AHA 2011.

In [67]:
d[2011]['HSACODE'] = pd.to_numeric(d[2011]['HSACODE'])
#d[2011]['HSACODE'].value_counts()

In [90]:
#d[2011]['HSACODE'].dtype

In [65]:
d[2011]['HSACODE'] = d[2011]['HSACODE'].replace("NaN", "")

Now I need to add HHI calculation for inpatient days and staffed beds to main dataframe. 

In [81]:
aha_subset = pd.merge(aha_subset, df)
aha_subset = pd.merge(aha_subset, a)

Let's get the summary statistics for staffed beds (BDTOT), inpatient days(IPDTOT), and HHI for Staffed bed and HHI for inpatient days. 

In [105]:
# Set ipython's max row display to 100
pd.set_option('display.max_row', 100)

# Set iPython's max column width to 60
pd.set_option('display.max_columns', 60)
aha_subset[['BDTOT', 'IPDTOT','HHI_HSA_BED', 'HHI_HSA_PDAYS']].describe()

,BDTOT,IPDTOT,HHI_HSA_BED,HHI_HSA_PDAYS
count,49506.000000,49506.000000,49506.000000,49506.000000
mean,151.548822,36250.115178,5974.122579,6081.613521
std,185.907219,51747.790984,3620.220429,3581.859570
min,1.000000,1.000000,383.720160,415.712238
25%,33.000000,5426.000000,2429.050673,2635.593025
50%,83.000000,17444.500000,5488.886120,5692.783701
75%,200.000000,46430.500000,10000.000000,10000.000000
max,2877.000000,761889.000000,10000.000000,10000.000000


Output below shows how HHI changes over the year in one HSA with 1001 code.

In [106]:
aha_subset.loc[aha_subset['HSACODE'] == 1001.0, ['ID', 'HSACODE','Year','BDTOT', 'HHI_HSA_BED', 'HHI_HSA_PDAYS']]

,ID,HSACODE,Year,BDTOT,HHI_HSA_BED,HHI_HSA_PDAYS
2999,6530010,1001.0,2010,192,7838.243573,9078.699404
3000,6530319,1001.0,2010,27,7838.243573,9078.699404
9194,6530010,1001.0,2011,266,8699.202895,9336.965221
9195,6530319,1001.0,2011,20,8699.202895,9336.965221
15364,6530010,1001.0,2012,206,6781.443423,8873.871035
15365,6530215,1001.0,2012,52,6781.443423,8873.871035
21556,6530010,1001.0,2013,247,7126.654064,8701.725407
21557,6530215,1001.0,2013,52,7126.654064,8701.725407
27736,6530010,1001.0,2014,211,6827.480519,8941.779014
27737,6530215,1001.0,2014,52,6827.480519,8941.779014


In [85]:
bins = [0, 20, 30, 40, np.inf]
names = [ 'Government-NonFederal', 'non-profit', 'for-profit', 'Government-Federal']
AHA1['ControlCode'] = pd.cut(AHA1['CNTRL'], bins, labels=names)

In [86]:
g = AHA1[AHA1['ControlCode'] == 'Government-NonFederal'].groupby('Year')

Now let's look at the number of non-profit, for-profit hospitals in the dataset.  

In [87]:
AHA1['ControlCode'].value_counts()

non-profit               25011
for-profit               13205
Government-NonFederal    10329
Government-Federal        1719
Name: ControlCode, dtype: int64

These number are not categorized based on year. For example number of non-profit hospitals over the years as follows:

In [89]:
AHA1['NonProfit'] = np.where(AHA1['ControlCode'] == 'non-profit',1, 0)
AHA1[AHA1['NonProfit'] == 1].groupby(level=[3]).size()

Year
2010    3141
2011    3140
2012    3119
2013    3146
2014    3104
2015    3099
2016    3107
2017    3155
dtype: int64

Church-operated hospitals are considered as non-profit. Double check if it is correct. [AHA website](https://www.aha.org/statistics/fast-facts-us-hospitals) shows that number of non-profit are little less than 3,000 in 2017. Let's exclude the church-based hospitals from the analysis. 

In [93]:
AHA1[AHA1['CNTRL'] == 23].groupby(level=[3]).size()

Year
2010    2595
2011    2595
2012    2581
2013    2609
2014    2570
2015    2574
2016    2596
2017    2641
dtype: int64

It shows that number of non-profit hospitals are less than the number at AHA website. 

**Notes**: find the reason of why there is a difference.

Now I am checking the teaching hospitals. According to AHA documentation, if a hospital is a member of Council of Teaching Hospital of the Association of American Medical Colleges (COTH), it is considered as major teaching hospitals. Following output shows the number of major teaching hospitals over the year.

In [82]:
AHA1[AHA1['MAPP8'] == 1].groupby(level=[3]).size()

Year
2010    365
2011    365
2012    323
2013    323
2014    302
2015    314
2016    303
2017    302
dtype: int64

The following outputs show that some hospitals are part of system but it is not coded in the data. For example, The University of Vermont Health Network is a hospital-system that has 4 hospitals in its system. However, it is not coded as system. It appears that for some years, the data includes only hospital name, but later years hospital name includes system name too. 
**Notes** Please check if there is any text-mining method to catch those hospitals. 

In [94]:
#AHA1[['MNAME', 'MLOCADDR']].loc["6211490"]

In [95]:
#AHA1[AHA1['MNAME'].str.contains("The University of Vermont Health Network", na=False)]

In [6]:
pd.set_option('display.max_colwidth', -1)
AHA.loc[AHA.ID == "6211490",['MNAME','SYSNAME', 'SYSID']]

,MNAME,SYSNAME,SYSID
413,Elizabethtown Community Hospital,NaN,NaN
408,Elizabethtown Community Hospital,NaN,NaN
403,Elizabethtown Community Hospital,NaN,NaN
397,Elizabethtown Community Hospital,NaN,NaN
393,The University of Vermont Health Network Elizabethtown Community Hospital,NaN,NaN
397,The University of Vermont Health Network Elizabethtown Community Hospital,NaN,NaN
394,The University of Vermont Health Network Elizabethtown Community Hospital,NaN,NaN
396,The University of Vermont Health Network Elizabethtown Community Hospital,NaN,NaN


In [96]:
#AHA['SYSNAME'].isnull().sum()

In [74]:
#mask = np.column_stack([AHA[col].str.contains("The University of Vermont Health Network", na=False) for col in AHA])

In [75]:
AHA1[['MNAME', 'MLOCADDR']].loc["6130015"]

MNAME  \
NPINUM       MCRNUM   Year                                                                            
1508845637.0 470001.0 2010  Central Vermont Medical Center                                            
1508845637   470001.0 2011  Central Vermont Medical Center                                            
1508845637.0 470001.0 2012  Central Vermont Medical Center                                            
             470001   2013  Central Vermont Medical Center                                            
                      2014  The University of Vermont Health Network Central Vermont Medical Center   
                      2015  The University of Vermont Health Network Central Vermont Medical Center   
                      2016  The University of Vermont Health Network Central Vermont Medical Center   
                      2017  The University of Vermont Health Network Central Vermont Medical Center   

                                   MLOCADDR  
NPINUM       MCRNUM   Year                   
1508845637.0 470001.0 2010  130 Fisher Road  
1508845637   470001.0 2011  130 Fisher Road  
1508845637.0 470001.0 2012  130 Fisher Road  
             470001   2013  130 Fisher Road  
                      2014  130 Fisher Road  
                      2015  130 Fisher Road  
                      2016  130 Fisher Road  
                      2017  130 Fisher Road

In [3]:
AHA1[['MNAME', 'MLOCADDR']].loc["6212350"]

MNAME  \
NPINUM       MCRNUM   Year                                                      
1114954682.0 330084.0 2010                          Alice Hyde Medical Center   
1114954682   330084.0 2011                          Alice Hyde Medical Center   
1114954682.0 330084.0 2012                          Alice Hyde Medical Center   
             330084   2013                          Alice Hyde Medical Center   
                      2014                          Alice Hyde Medical Center   
                      2015  The University of Vermont Health Network - Ali...   
                      2016  The University of Vermont Health Network - Ali...   
                      2017  The University of Vermont Health Network - Ali...   

                                   MLOCADDR  
NPINUM       MCRNUM   Year                   
1114954682.0 330084.0 2010  133 Park Street  
1114954682   330084.0 2011  133 Park Street  
1114954682.0 330084.0 2012  133 Park Street  
             330084   2013  133 Park Street  
                      2014  133 Park Street  
                      2015  133 Park Street  
                      2016  133 Park Street  
                      2017  133 Park Street

In [8]:
AHA1.loc[AHA1.MNAME.str.startswith('The University of Vermont'), ['MNAME', 'MLOCADDR', 'SYSNAME', 'SYSID']]

,,,,MNAME,MLOCADDR,SYSNAME,SYSID
ID,NPINUM,MCRNUM,Year,,,,
6130001,1.568420e+09,470003,2014,The University of Vermont Health Network University of Vermont Medical Center,111 Colchester Avenue,NaN,NaN
6130015,1.508846e+09,470001,2014,The University of Vermont Health Network Central Vermont Medical Center,130 Fisher Road,NaN,NaN
6211490,1.891785e+09,331302,2014,The University of Vermont Health Network Elizabethtown Community Hospital,Park Street,NaN,NaN
6213990,1.033271e+09,330250,2014,The University of Vermont Health Network-Champlain Valley Physicians Hospital,75 Beekman Street,NaN,NaN
6130001,1.568420e+09,470003,2015,The University of Vermont Health Network University of Vermont Medical Center,111 Colchester Avenue,NaN,NaN
6130015,1.508846e+09,470001,2015,The University of Vermont Health Network Central Vermont Medical Center,130 Fisher Road,NaN,NaN
6211490,1.891785e+09,331302,2015,The University of Vermont Health Network Elizabethtown Community Hospital,75 Park Street,NaN,NaN
6212350,1.114955e+09,330084,2015,The University of Vermont Health Network - Alice Hyde Medical Center,133 Park Street,NaN,NaN
6213990,1.033271e+09,330250,2015,The University of Vermont Health Network-Champlain Valley Physicians Hospital,75 Beekman Street,NaN,NaN


In [98]:
#AHA1[['MNAME', 'MLOCADDR', 'SYSNAME', 'SYSID']].loc["6110367"]

In [97]:
#AHA1[['MNAME', 'MLOCADDR', 'SYSNAME', 'SYSID']].loc["6110160"]

The number of hospitals that part of system over the years. 

In [101]:
AHA1[AHA1['MHSMEMB'] == 1.0].groupby(level=[3]).size()

Year
2010    3671
2011    3735
2012    3811
2013    3884
2014    3936
2015    4004
2016    4056
2017    4108
dtype: int64